#### Try to convert input sentences into numeric sequences

In [1]:
import tensorflow as tf

Korpus merupakan kumpulan teks yang tersimpan secara elektronik untuk berbagai kebutuhan penyelidikan dan penelitian, salah satunya dalam pemrosesan bahasa alami (Natural Language Processing).

In previous lab, i use the TextVectorization layer to build vocabulary from my corpus. Itu menghasilkan daftar di mana kata-kata yang lebih sering muncul memiliki indeks yang lebih rendah.

In [3]:
#Sample inputs

sentences=[
    'Ayu cantik',
    'Wahyuni bukan Wahyu',
    'I love me',
    'Am I supernova?'
]

#inisialisasi layer
vectorize_layer=tf.keras.layers.TextVectorization()

#Build vocabulary
#tiap kata pada sentences disimpan dalam bentuk indeks
vectorize_layer.adapt(sentences)

#Get the vocabulary
vocabulary=vectorize_layer.get_vocabulary()

#Print the token index
for index, word in enumerate(vocabulary):
    print(index, word)

0 
1 [UNK]
2 i
3 wahyuni
4 wahyu
5 supernova
6 me
7 love
8 cantik
9 bukan
10 ayu
11 am


In [9]:
#Memeriksa token pada suatu kalimat
#convert sentence to integer sequences

sample_sentence=[
        'saya makan nasi',
        'i love me',
        'i love nasi'
]

#convert
sequences=vectorize_layer(sample_sentence)

# for index, word in enumerate(sample_sentence):
print(sequences)

tf.Tensor(
[[1 1 1]
 [2 7 6]
 [2 7 1]], shape=(3, 3), dtype=int64)
